In [ ]:
#!pip install mat73

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr
import rioxarray as rxr
import scipy.io as sio
import datetime
import mat73

---
**Thermal infrared images**

Filepath list of .mat files

In [ ]:
ir_files = [r'C:\Users\steve\OneDrive\Documents\School Stuff\UW\Mountain Hydrology Research Group\IR_PLANE_PROCESSED-20200908T002540Z-001\IR_PLANE_PROCESSED\SNOWEX2020_IR_PLANE_2020Feb08_mosaicked.mat',
            r'C:\Users\steve\OneDrive\Documents\School Stuff\UW\Mountain Hydrology Research Group\IR_PLANE_PROCESSED-20200908T002540Z-001\IR_PLANE_PROCESSED\SNOWEX2020_IR_PLANE_2020Feb10_mosaicked.mat',
            r'C:\Users\steve\OneDrive\Documents\School Stuff\UW\Mountain Hydrology Research Group\IR_PLANE_PROCESSED-20200908T002540Z-001\IR_PLANE_PROCESSED\SNOWEX2020_IR_PLANE_2020Feb11_mosaicked.mat',
            r'C:\Users\steve\OneDrive\Documents\School Stuff\UW\Mountain Hydrology Research Group\IR_PLANE_PROCESSED-20200908T002540Z-001\IR_PLANE_PROCESSED\SNOWEX2020_IR_PLANE_2020Feb12_mosaicked.mat'
            ]

Load a .mat file from the filepath list.

In [ ]:
aircraft_ir_data_mat = sio.loadmat(ir_files[0])  

Display the contents of what we have loaded. This is a dictionary containing metadata and numpy arrays of the data variables.

In [ ]:
aircraft_ir_data_mat

Create an xarray dataset with this data

In [ ]:
# convert time in epoch to isoformat
time = [datetime.datetime.utcfromtimestamp(this_time).isoformat() for this_time in aircraft_ir_data_mat['time'].squeeze()]


# create dataset
ds = xr.Dataset({
    'STCtemp': xr.DataArray(
                data   = aircraft_ir_data_mat['STCtemp'], # scaled IR temperature data
                dims   = ['y', 'x', 'time'],
                coords = {'x': aircraft_ir_data_mat['Eutm'].squeeze(), # easting coordinates (MGRS-UTM, m
                          'y': aircraft_ir_data_mat['Nutm'].squeeze(), # northing grid coordinates (MGRS-UTM, 
                          'time': time}, # time in isoformat

                attrs  = {
                    '_FillValue'  : np.nan,
                    'units'       : 'Celsius',
                    'description' : 'land and snow surface brightness temperature - no emissivity correction',
                    'timezone'    : 'time in UTC'
                    }
                ),
    #'zDEM': xr.DataArray(
    #            data   = aircraft_data_mat['zDEM'], # elevation (meters)
    #            dims   = ['y', 'x'],
    #            coords = {'x': aircraft_data_mat['Eutm'].squeeze(),
    #                      'y': aircraft_data_mat['Nutm'].squeeze()},
    #            attrs  = {
    #                '_FillValue'  : np.nan,
    #                'units'       : 'meters',
    #                'description' : 'digital elevation map used for georectification',
    #                'source'      : 'https://doi.org/10.5066/F7PR7TFT',
    #                'horizontal datum' : 'WGS84',
    #                'vertical datum' : 'EGM96',
    #                'original resolution' : '30 m / 1 arc-second'
    #                }
    #        )
},
        attrs = {'description': 'Airborne thermal infrared imagery from University of Washington Applied Physics Lab Compact Airborne System for Imaging the Environment instrument. Imagery of Grand Mesa, Colorado from SnowEx 2020 field campaign.'}
    )



Specify our spatial dimensions and coordinate reference system

In [ ]:
ds.rio.set_spatial_dims('y', 'x', inplace=True)
ds.rio.write_crs("epsg:26913", inplace=True) # UTM Zone 13N / NAD83
ds = ds.transpose('time', 'y', 'x').rio.reproject("epsg:32612")

For each timestamp, save a separate netcdf of geotiff file.

In [ ]:
for t, _ in enumerate(time):
    time_str = time[t].replace(':','').split('.')[0]
    ds.isel(time=t).to_netcdf('SNOWEX2020_IR_PLANE_2020Feb08_mosaicked_{}.nc'.format(time_str))
    ds.isel(time=t).rio.to_raster('SNOWEX2020_IR_PLANE_2020Feb08_mosaicked_{}.tif'.format(time_str))

---

**Visible images**

Filepath list of .mat files

In [ ]:
vis_files = [r'C:\Users\steve\OneDrive\Documents\School Stuff\UW\Mountain Hydrology Research Group\IR_PLANE_PROCESSED-20200908T002540Z-001\SNOWEX2020_EO_PLANE_2020Feb08_mosaicked.mat']

Load a .mat file from the filepath list. (need to use the [mat73](https://github.com/skjerns/mat7.3) library for this version of matlab file)

In [ ]:
aircraft_vis_data_mat = mat73.loadmat(vis_files[0])

Display the contents of what we have loaded. This is a dictionary containing metadata and numpy arrays of the data variables.

In [ ]:
aircraft_vis_data_mat

Create an xarray dataset with this data

In [ ]:
# convert time in epoch to isoformat
time = [datetime.datetime.utcfromtimestamp(this_time).isoformat() for this_time in aircraft_vis_data_mat['time'].squeeze()]

# scale image color data to between 0 and 1
#original_max = np.nanmax(aircraft_vis_data_mat['SRGB'])
#original_min = np.nanmin(aircraft_vis_data_mat['SRGB'])
#scaled_RGB_data = (aircraft_vis_data_mat['SRGB'] - original_min) / (original_max - original_min) * 255
#scaled_RGB_data = scaled_RGB_data.astype('int')

# create dataset
ds = xr.Dataset({
    'SRGB': xr.DataArray(
                data   = aircraft_vis_data_mat['SRGB'], # Red, Green, Blue image bands
                dims   = ['y', 'x', 'band', 'time'],
                coords = {'x': aircraft_vis_data_mat['Xs'][0,:], # easting coordinates (MGRS-UTM, m)
                          'y': aircraft_vis_data_mat['Ys'][:,0], # northing grid coordinates (MGRS-UTM, m)
                          'band': ['r', 'g', 'b'], # labels for red, green, blue image bands
                          'time': time}, # time in isoformat

                attrs  = {
                    '_FillValue'  : np.nan,
                    'units'       : 'digital numbers',
                    'description' : 'visible imagery',
                    'timezone'    : 'time in UTC'
                    }
                ),
    #'zDEM': xr.DataArray(
    #            data   = aircraft_data_mat['zDEM'], # elevation (meters)
    #            dims   = ['y', 'x'],
    #            coords = {'x': aircraft_data_mat['Eutm'].squeeze(),
    #                      'y': aircraft_data_mat['Nutm'].squeeze()},
    #            attrs  = {
    #                '_FillValue'  : np.nan,
    #                'units'       : 'meters',
    #                'description' : 'digital elevation map used for georectification',
    #                'source'      : 'https://doi.org/10.5066/F7PR7TFT',
    #                'horizontal datum' : 'WGS84',
    #                'vertical datum' : 'EGM96',
    #                'original resolution' : '30 m / 1 arc-second'
    #                }
    #        )
},
        attrs = {'description': 'Airborne visible imagery from University of Washington Applied Physics Lab Compact Airborne System for Imaging the Environment instrument. Imagery of Grand Mesa, Colorado from SnowEx 2020 field campaign.'}
    )



Specify our spatial dimensions and coordinate reference system

In [ ]:
ds.rio.set_spatial_dims('y', 'x', inplace=True)
ds.rio.write_crs("epsg:26913", inplace=True) # UTM Zone 13N / NAD83

For each timestamp, save a separate netcdf or geotiff file.

In [ ]:
for t, _ in enumerate(time):
    time_str = time[t].replace(':','').split('.')[0]
    ds.isel(time=t,band=2).transpose('y', 'x').rio.reproject("epsg:32612").to_netcdf('SNOWEX2020_EO_PLANE_2020Feb08_mosaicked_{}.nc'.format(time_str))
    _ds = ds.isel(time=t,band=2).SRGB.transpose('y', 'x').rio.reproject("epsg:32612")
    _ds.rio.to_raster('SNOWEX2020_EO_PLANE_2020Feb08_mosaicked_{}.tif'.format(time_str))